# Data import

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.font_manager as fm
import seaborn as sns
import datetime as dt
import matplotlib.font_manager as fm
import random
import pickle
import datetime as dt
import itertools
import os
import holidays
import holidays


from matplotlib import gridspec
from scipy import stats
from prophet import Prophet
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
from prophet.plot import plot_cross_validation_metric
from prophet.plot import add_changepoints_to_plot
from prophet.plot import plot_plotly, plot_components_plotly
from prophet.plot import add_changepoints_to_plot
from plotnine import *
from pathlib import Path


In [10]:
# BASE_DIR = Path(__name__).resolve().parent.parent
FILES_DIR = '../files/'
bidding_df = pd.read_csv(FILES_DIR + 'bidding_results.csv')

In [11]:
fm.get_fontconfig_fonts()
font_location = 'C:\\WINDOWS\\Fonts\\malgun.ttf' # For Windows
font_name = fm.FontProperties(fname=font_location).get_name()
matplotlib.rc('font', family=font_name)

C:\Users\bongkyun\AppData\Local\Temp\ipykernel_8744\2582751676.py:1: MatplotlibDeprecationWarning: 
The get_fontconfig_fonts function was deprecated in Matplotlib 3.5 and will be removed two minor releases later.


In [12]:
bidding = bidding_df.copy()

# Null Value Processing

In [13]:
# column describe 
def descriptive_info(bidding):
    number_of_cells = bidding.count(axis=0, level=None, numeric_only=False)
    print('Total number of cells the datafile has: \n' + str(number_of_cells))
    print('\n')
    print('Printing descriptive statistics...\n\n', bidding.describe())

descriptive_info(bidding)

Total number of cells the datafile has: 
index              1337088
Impression         1337088
Click              1337088
Cost               1337088
Sum of AD rank     1337078
Bid                1337088
Price(PC)          1337088
Price(Mobile)      1337088
Delivery Fee       1337088
Category Name 1    1337088
Category Name 2    1337088
Category Name 3    1331221
Category Name 4     930132
date               1337088
dtype: int64


Printing descriptive statistics...

               index    Impression         Click          Cost  Sum of AD rank  \
count  1.337088e+06  1.337088e+06  1.337088e+06  1.337088e+06    1.337078e+06   
mean   5.167098e+06  4.412753e+01  7.189467e-01  1.800053e+02    1.138271e+01   
std    2.655813e+06  3.434510e+02  3.423456e+00  1.772706e+03    1.323231e+01   
min    2.578600e+04  0.000000e+00  0.000000e+00  0.000000e+00    1.000000e+00   
25%    3.300168e+06  3.000000e+00  0.000000e+00  0.000000e+00    3.706897e+00   
50%    4.637081e+06  1.000000e+01  0.000000

In [14]:
# bidding 날짜 datetime으로 만들기
def _trans_date_time(bidding):
    bidding.index = pd.to_datetime(bidding['date'], format='%Y%m%d')
    bidding.drop('date', axis=1, inplace=True)
    return bidding

_trans_date_time(bidding)

,index,Impression,Click,Cost,Sum of AD rank,Bid,Price(PC),Price(Mobile),Delivery Fee,Category Name 1,Category Name 2,Category Name 3,Category Name 4
date,,,,,,,,,,,,,
2022-10-31,3575092,3,0,0.0,3.000000,50,165900,165900,0,가구/인테리어,침실가구,장롱/붙박이장,드레스룸
2022-10-31,3575637,6,0,0.0,3.333333,50,10900,10900,0,가구/인테리어,인테리어소품,아로마/캔들용품,아로마방향제/디퓨저
2022-10-31,4129259,7,0,0.0,8.000000,50,255000,255000,-1,가구/인테리어,거실가구,소파,패브릭소파
2022-10-31,4131479,36,3,150.0,4.416667,140,190000,190000,20000,가구/인테리어,아동/주니어가구,책상의자세트,NaN
2022-10-31,4131482,224,0,0.0,5.696429,50,129000,129000,0,가구/인테리어,아동/주니어가구,책상,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-03,2566795,5,0,0.0,13.200000,500,19900,19900,0,가구/인테리어,인테리어소품,아로마/캔들용품,아로마방향제/디퓨저
2023-01-03,2566880,3,0,0.0,17.000000,500,23900,23900,3000,생활/건강,문구/사무용품,이벤트/파티용품,데코용품
2023-01-03,2566881,1,0,0.0,8.000000,500,2500,2500,3000,가구/인테리어,인테리어소품,기타장식용품,NaN


In [15]:
# CTR = 클릭률
# CPC = 클릭당 비용
# CPM = 1000회 노출당 비용
# Sum of AD rank = 반올림
bidding["CTR"] = (bidding["Click"]/bidding["Impression"]) * 100
bidding["CPC"] = (bidding["Cost"]/bidding["Click"])
bidding["CPM"] = (bidding["Click"]/bidding["Cost"])
bidding['Sum of AD rank'] = bidding['Sum of AD rank'].round(0)

In [16]:
bidding_drop_null = bidding.copy()
bidding_drop_null['Category Name 4'].fillna('null_cat4', inplace = True)
bidding_drop_null['CPC'].fillna(0, inplace = True)
bidding_drop_null['CCC'].fillna(0, inplace = True)
bidding_drop_null.dropna(axis = 0, inplace = True)
print(bidding_drop_null.isnull().sum().sort_values(ascending=False)) # null값 퍼센트로 표현

KeyError: 'CCC'

In [ ]:
bidding_drop_null

# Data pickle load

In [ ]:
# # save and compress.
# with open('bidding_drop_null.pickle', 'wb') as f:
#     pickle.dump(bidding_drop_null, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
# # load
# with open('bidding_drop_null.pickle', 'rb') as f:
#     bidding_drop_null = pickle.load(f)

In [ ]:
# bidding_drop_null

# Price 구간별 Cost 민감도에 따른 Rank

In [ ]:
price_down_5 = bidding_drop_null[bidding_drop_null['Price(PC)'] <= 50000]

price_5_10 = bidding_drop_null[(bidding_drop_null['Price(PC)'] > 50000) & (bidding_drop_null['Price(PC)'] < 100000)]

price_10_15 = bidding_drop_null[(bidding_drop_null['Price(PC)'] > 100000) & (bidding_drop_null['Price(PC)'] < 150000)]

price_15_20 = bidding_drop_null[(bidding_drop_null['Price(PC)'] > 150000) & (bidding_drop_null['Price(PC)'] < 200000)]

price_up_20 = bidding_drop_null[bidding_drop_null['Price(PC)'] >= 200000]

In [ ]:
def price_Analysis(df):
        
    print('비용평균')
    print(df['Cost'].mean())
    print('-' * 50)
    print('\n')
    
    cost_non_zero = df[df['Cost'] != 0]
    
    
    print(f'코스트 썼을때 평균 순위 : {cost_non_zero["Sum of AD rank"].mean()}')

    cost_zero = df[df['Cost'] == 0]

    print(f'코스트 안 썼을때 평균 순위 : {cost_zero["Sum of AD rank"].mean()}')
    print('-' * 50)
    print('\n')
  

    print("광고숫자비율")
    print((len(list(set(df['index'].values))) / len(df))*100)
    print('-' * 50)
    print('\n')
    
    
    rank_del_zero = df[df['Delivery Fee'] <= 0]
    rank_del_non_zero = df[df['Delivery Fee'] > 0]
    print('배달비 없음 비율')
    print((rank_del_zero.shape[0] / df['Delivery Fee'].shape[0]*100))
    print('\n')
    print('배달비 있음 비율')
    print((rank_del_non_zero.shape[0] / df['Delivery Fee'].shape[0]*100))  
    print('-' * 50)
    print('\n')
    

    print('코스트비율')
    print(df['Cost'].value_counts(normalize=True))
    print('-' * 50)
    print('\n')
    
    plt.figure(figsize=(28, 12))
    plt.subplot(4, 1, 1)
    print('코스트')
    display(cost_non_zero['Cost'].describe())
    plt.hist(cost_non_zero['Cost'], bins=20)
    plt.title('cost_non_zero')
    plt.xticks()
    
    
    print('-' * 50)
    print('\n')
    
    
    plt.subplot(4, 1, 2)
    print('카테고리1')
    display(df['Category Name 1'].value_counts(normalize=True))
    sns.countplot(x=df['Category Name 1'], data=df)
    plt.title('Category Name 1')
    plt.xticks()
    
    
    print('-' * 50)
    print('\n')
    
    
    plt.subplot(4, 1, 3)
    print('코스트 있음 , 순위')
    display(cost_non_zero['Sum of AD rank'].describe())
    plt.hist(cost_non_zero['Sum of AD rank'], bins=20)
    plt.title('cost_non_zero')
    plt.xticks()
    
    
    print('-' * 50)
    print('\n')
    
    
    plt.subplot(4, 1, 4)
    print('코스트 없음 , 순위')
    display(cost_zero['Sum of AD rank'].describe())
    plt.hist(cost_zero['Sum of AD rank'], bins=20)
    plt.title('cost_zero')
    plt.xticks()

    

In [ ]:
# 수치표현
# 그래프표현
# text = 5만원 이하 상품 가격 데이터 특징
price_Analysis(price_down_5)

In [ ]:
# 수치표현
# 그래프표현
# text = 5~10만원 상품 가격 데이터 특징
price_Analysis(price_5_10)

In [ ]:
# 수치표현
# 그래프표현
# text = 10~15만원 상품 가격 데이터 특징
price_Analysis(price_10_15)

In [ ]:
# 수치표현
# 그래프표현
# text = 15~20만원 상품 가격 데이터 특징
price_Analysis(price_15_20)

In [ ]:
# 수치표현
# 그래프표현
# text = 20만원 이상 상품 가격 데이터 특징
price_Analysis(price_up_20)

In [ ]:
def corr_heat(df):
    sns.set(rc = {'figure.figsize':(15,8)}, font_scale = 1)
    sns.heatmap(data = df.corr(), annot=True, linewidths=1)

In [ ]:
# 그래프 표현
# text = 5만원 이하 상품 가격 데이터 상관관계
corr_heat(price_down_5)

In [ ]:
# 그래프 표현
# text = 5~10만원 상품 가격 데이터 상관관계
corr_heat(price_5_10)

In [ ]:
# 그래프 표현
# text = 10~15만원 상품 가격 데이터 상관관계
corr_heat(price_10_15)

In [ ]:
# 그래프 표현
# text = 15~20만원 상품 가격 데이터 상관관계
corr_heat(price_15_20)

In [ ]:
# 그래프 표현
# text = 20만원 이상 상품 가격 데이터 상관관계
corr_heat(price_up_20)

# Rank 구간별

In [ ]:
# 네이버 페이지 기본 default 40, 한페이지에 광고가 6개 상위3개 중간에3개(20개보기, 40개보기, 60개보기, 80개보기)
# log에서 얻은 인사이트로 윈도우가 압도적으로 많으니 윈도우 기준

In [ ]:
rank_down_26 = bidding_drop_null[bidding_drop_null['Sum of AD rank'] <= 26]

rank_26_46 = bidding_drop_null[(bidding_drop_null['Sum of AD rank'] > 26) & (bidding_drop_null['Sum of AD rank'] < 46)]

rank_46_66 = bidding_drop_null[(bidding_drop_null['Sum of AD rank'] > 46) & (bidding_drop_null['Sum of AD rank'] < 66)]

rank_66_86 = bidding_drop_null[(bidding_drop_null['Sum of AD rank'] > 66) & (bidding_drop_null['Sum of AD rank'] < 86)]

rank_up_86 = bidding_drop_null[bidding_drop_null['Sum of AD rank'] >= 86]

In [ ]:
def rank_col(df):
    print("광고숫자비율")
    print((len(list(set(df['index'].values))) / len(df))*100)
    print('-' * 50)
    
    columns = ['Click', 'Cost', 'Impression', 'Price(PC)']
    
    for col in columns:
        print('\n')
        print(col)
        print(df[col].mean())
        print('-' * 50)
        print(df[col].value_counts(normalize=True))

In [ ]:
#수치표현
# text = Rank 구간별 광고 비율
print("rank_down_26 광고숫자비율")
print((len(list(set(rank_down_26['index'].values))) / len(rank_down_26))*100)
print('-' * 50)
print("rank_26_46 광고숫자비율")
print((len(list(set(rank_26_46['index'].values))) / len(rank_26_46))*100)
print('-' * 50)
print("rank_46_66 광고숫자비율")
print((len(list(set(rank_46_66['index'].values))) / len(rank_46_66))*100)
print('-' * 50)
print("rank_66_86 광고숫자비율")
print((len(list(set(rank_66_86['index'].values))) / len(rank_66_86))*100)
print('-' * 50)
print("rank_up_86 광고숫자비율")
print((len(list(set(rank_up_86['index'].values))) / len(rank_up_86))*100)
print('-' * 50)

In [ ]:
#수치표현
# text = Rank 구간별 광고 비율
print("rank_down_26 광고숫자비율")
print((len(list(set(rank_down_26['index'].values)))))
print('-' * 50)
print("rank_26_46 광고숫자비율")
print((len(list(set(rank_26_46['index'].values)))))
print('-' * 50)
print("rank_46_66 광고숫자비율")
print((len(list(set(rank_46_66['index'].values)))))
print('-' * 50)
print("rank_66_86 광고숫자비율")
print((len(list(set(rank_66_86['index'].values)))))
print('-' * 50)
print("rank_up_86 광고숫자비율")
print((len(list(set(rank_up_86['index'].values)))))
print('-' * 50)

In [ ]:
#수치표현
# text = Rank 구간별 Click 평균
print('rank_down_26')
print(rank_down_26['Click'].mean())
print('-' * 50)
print(rank_down_26['Click'].value_counts(normalize=True))
print('-' * 50)
print('rank_26_46')
print(rank_26_46['Click'].mean())
print('-' * 50)
print(rank_26_46['Click'].value_counts(normalize=True))
print('-' * 50)
print('rank_46_66')
print(rank_46_66['Click'].mean())
print('-' * 50)
print(rank_46_66['Click'].value_counts(normalize=True))
print('-' * 50)
print('rank_66_86')
print(rank_66_86['Click'].mean())
print('-' * 50)
print(rank_66_86['Click'].value_counts(normalize=True))
print('-' * 50)
print('rank_up_86')
print(rank_up_86['Click'].mean())
print('-' * 50)
print(rank_up_86['Click'].value_counts(normalize=True))
print('-' * 50)

In [ ]:
#수치표현
# text = Rank 구간별 Cost 평균
print('rank_down_26')
print(rank_down_26['Cost'].mean())
print('-' * 50)
print(rank_down_26['Cost'].value_counts(normalize=True))
print('-' * 50)
print('rank_26_46')
print(rank_26_46['Cost'].mean())
print('-' * 50)
print(rank_26_46['Cost'].value_counts(normalize=True))
print('-' * 50)
print('rank_46_66')
print(rank_46_66['Cost'].mean())
print('-' * 50)
print(rank_46_66['Cost'].value_counts(normalize=True))
print('-' * 50)
print('rank_66_86')
print(rank_66_86['Cost'].mean())
print('-' * 50)
print(rank_66_86['Cost'].value_counts(normalize=True))
print('-' * 50)
print('rank_up_86')
print(rank_up_86['Cost'].mean())
print('-' * 50)
print(rank_up_86['Cost'].value_counts(normalize=True))
print('-' * 50)

In [ ]:
#수치표현
# text = Rank 구간별 Impression 평균
print('rank_down_26')
print(rank_down_26['Impression'].mean())
print('-' * 50)
print(rank_down_26['Impression'].value_counts(normalize=True))
print('-' * 50)
print('rank_26_46')
print(rank_26_46['Impression'].mean())
print('-' * 50)
print(rank_26_46['Impression'].value_counts(normalize=True))
print('-' * 50)
print('rank_46_66')
print(rank_46_66['Impression'].mean())
print('-' * 50)
print(rank_46_66['Impression'].value_counts(normalize=True))
print('-' * 50)
print('rank_66_86')
print(rank_66_86['Impression'].mean())
print('-' * 50)
print(rank_66_86['Impression'].value_counts(normalize=True))
print('-' * 50)
print('rank_up_86')
print(rank_up_86['Impression'].mean())
print('-' * 50)
print(rank_up_86['Impression'].value_counts(normalize=True))
print('-' * 50)

In [ ]:
#수치표현
#mean으로 할지 median으로 할지는 멘토링후 결정
# text = Rank 구간별 Price(PC) 평균
print('rank_down_26')
print(rank_down_26['Price(PC)'].mean())
print('-' * 50)
print(rank_down_26['Price(PC)'].value_counts(normalize=True))
print('-' * 50)
print('rank_26_46')
print(rank_26_46['Price(PC)'].mean())
print('-' * 50)
print(rank_26_46['Price(PC)'].value_counts(normalize=True))
print('-' * 50)
print('rank_46_66')
print(rank_46_66['Price(PC)'].mean())
print('-' * 50)
print(rank_46_66['Price(PC)'].value_counts(normalize=True))
print('-' * 50)
print('rank_66_86')
print(rank_66_86['Price(PC)'].mean())
print('-' * 50)
print(rank_66_86['Price(PC)'].value_counts(normalize=True))
print('-' * 50)
print('rank_up_86')
print(rank_up_86['Price(PC)'].mean())
print('-' * 50)
print(rank_up_86['Price(PC)'].value_counts(normalize=True))
print('-' * 50)

In [ ]:
# median으로 한다면 그래프표현
# text = 상품가격 Boxplot
plt.figure(figsize=(16, 6))
plt.boxplot(bidding_drop_null['Cost'])
plt.title('Price')
plt.xticks(rotation=90)

In [ ]:
# 수치표현
# text = Rank 구간별 Price(PC) 중간값
print('rank_down_26')
print(rank_down_26['Price(PC)'].median())
print('-' * 50)
print(rank_down_26['Price(PC)'].value_counts(normalize=True))
print('-' * 50)
print('rank_26_46')
print(rank_26_46['Price(PC)'].median())
print('-' * 50)
print(rank_26_46['Price(PC)'].value_counts(normalize=True))
print('-' * 50)
print('rank_46_66')
print(rank_46_66['Price(PC)'].median())
print('-' * 50)
print(rank_46_66['Price(PC)'].value_counts(normalize=True))
print('-' * 50)
print('rank_66_86')
print(rank_66_86['Price(PC)'].median())
print('-' * 50)
print(rank_66_86['Price(PC)'].value_counts(normalize=True))
print('-' * 50)
print('rank_up_86')
print(rank_up_86['Price(PC)'].median())
print('-' * 50)
print(rank_up_86['Price(PC)'].value_counts(normalize=True))
print('-' * 50)

## Rank 26이하와 26초과로 분석

In [ ]:
rank_down_26 = bidding_drop_null[bidding_drop_null['Sum of AD rank'] <= 26]
rank_up_26 = bidding_drop_null[bidding_drop_null['Sum of AD rank'] > 26]

In [ ]:
def rank_Analysis(df):
        
    print('비용평균')
    print(df['Cost'].mean())
    print('-' * 50)
    print('\n')
    
    cost_non_zero = df[df['Cost'] != 0]
    
    
    print(f'코스트 썼을때 평균 순위 : {cost_non_zero["Sum of AD rank"].mean()}')

    cost_zero = df[df['Cost'] == 0]

    print(f'코스트 안 썼을때 평균 순위 : {cost_zero["Sum of AD rank"].mean()}')
    print('-' * 50)
    print('\n')
  

    print("광고숫자비율")
    print((len(list(set(df['index'].values))) / len(df))*100)
    print('-' * 50)
    print('\n')
    
    
    rank_del_zero = df[df['Delivery Fee'] <= 0]
    rank_del_non_zero = df[df['Delivery Fee'] > 0]
    print('배달비 없음 비율')
    print((rank_del_zero.shape[0] / df['Delivery Fee'].shape[0]*100))
    print('\n')
    print('배달비 있음 비율')
    print((rank_del_non_zero.shape[0] / df['Delivery Fee'].shape[0]*100))  
    print('-' * 50)
    print('\n')
    
    print('상품가격 중간값')
    print(f'{df["Price(PC)"].median()} 원')
    print('-' * 50)
    print('\n')
    
    plt.figure(figsize=(28, 12))
    print('카테고리1')
    display(df['Category Name 1'].value_counts(normalize=True))
    sns.countplot(x=df['Category Name 1'], data=df)
    plt.title('Category Name 1')
    plt.xticks(rotation=90)
    

In [ ]:
#수치표현
#그래프표현
# text = Rank 26 이하 특징
rank_Analysis(rank_down_26)

In [ ]:
#수치표현
#그래프표현
# text = Rank 26 초과 특징
rank_Analysis(rank_up_26)

In [ ]:
#그래프표현
# text = Rank 26 이하 상관관계
corr_heat(rank_down_26)

In [ ]:
#그래프표현
# text = Rank 26 초과 상관관계
corr_heat(rank_up_26)

In [ ]:
rank_down_7 = bidding_drop_null[bidding_drop_null['Sum of AD rank'] <= 7]

In [ ]:
#수치표현
#그래프표현
# text = Rank 7 이하 특징
rank_Analysis(rank_down_7)

In [ ]:
#그래프표현
# text = Rank 7 이하 상관관계
corr_heat(rank_down_7)

# Category 1별 분석

In [ ]:
#수치표현
bidding_drop_null['Category Name 1'].value_counts(normalize=True)

In [ ]:
cat_val_1 = bidding_drop_null[bidding_drop_null['Category Name 1'].isin(['식품'])]
cat_val_2 = bidding_drop_null[bidding_drop_null['Category Name 1'].isin(['스포츠/레저'])]
cat_val_3 = bidding_drop_null[bidding_drop_null['Category Name 1'].isin(['출산/육아'])]
cat_val_4 = bidding_drop_null[bidding_drop_null['Category Name 1'].isin(['패션의류'])]
cat_val_5 = bidding_drop_null[bidding_drop_null['Category Name 1'].isin(['생활/건강'])]
cat_val_6 = bidding_drop_null[bidding_drop_null['Category Name 1'].isin(['디지털/가전'])]
cat_val_7 = bidding_drop_null[bidding_drop_null['Category Name 1'].isin(['가구/인테리어'])]
cat_val_8 = bidding_drop_null[bidding_drop_null['Category Name 1'].isin(['화장품/미용'])]
cat_val_9 = bidding_drop_null[bidding_drop_null['Category Name 1'].isin(['패션잡화'])]
cat_val_10 = bidding_drop_null[bidding_drop_null['Category Name 1'].isin(['여가/생활편의'])]

In [ ]:
def cat_val(df):
    
    print(f'광고비율 : {(len(list(set(df["index"].values))) / len(df))*100}')
    print('-' * 50)
    cat_col = ['Click', 'Cost', 'Sum of AD rank', 'Impression']
    
    for col in cat_col:
        print(f'{col} 평균 : {df[col].mean()}')
        print('-' * 50)
        print(df[col].value_counts(normalize=True))
        print('-' * 50)
        plt.figure(figsize=(16, 6))
        plt.hist(df[col], bins=20)
        plt.title(col)
        plt.xticks(rotation=90)

In [ ]:
#수치표현
# Total Data 광고 비율

print((len(list(set(bidding_drop_null['index'].values))) / len(bidding_drop_null))*100)

In [ ]:
#수치표현
#그래프표현
# text = Category1 별 광고 비율
print("식품 광고숫자비율")
print((len(list(set(cat_val_1['index'].values))) / len(cat_val_1))*100)
print('-' * 50)
print("스포츠/레저 광고숫자비율")
print((len(list(set(cat_val_2['index'].values))) / len(cat_val_2))*100)
print('-' * 50)
print("출산/육아 광고숫자비율")
print((len(list(set(cat_val_3['index'].values))) / len(cat_val_3))*100)
print('-' * 50)
print("패션의류 광고숫자비율")
print((len(list(set(cat_val_4['index'].values))) / len(cat_val_4))*100)
print('-' * 50)
print("생활/건강 광고숫자비율")
print((len(list(set(cat_val_5['index'].values))) / len(cat_val_5))*100)
print('-' * 50)
print("디지털/가전 광고숫자비율")
print((len(list(set(cat_val_6['index'].values))) / len(cat_val_6))*100)
print('-' * 50)
print("가구/인테리어 광고숫자비율")
print((len(list(set(cat_val_7['index'].values))) / len(cat_val_7))*100)
print('-' * 50)
print("화장품/미용 광고숫자비율")
print((len(list(set(cat_val_8['index'].values))) / len(cat_val_8))*100)
print('-' * 50)
print("패션잡화 광고숫자비율")
print((len(list(set(cat_val_9['index'].values))) / len(cat_val_9))*100)
print('-' * 50)
print("여가/생활편의 광고숫자비율")
print((len(list(set(cat_val_10['index'].values))) / len(cat_val_10))*100)
print('-' * 50)

In [ ]:
#수치표현
# text = Total Data Click 평균

bidding_drop_null['Click'].mean()

In [ ]:
#수치표현
#그래프표현
# text = Category1 별 Click 평균
print('식품')
print(cat_val_1['Click'].mean())
print('-' * 50)
print(cat_val_1['Click'].value_counts(normalize=True))
print('-' * 50)
print('스포츠/레저')
print(cat_val_2['Click'].mean())
print('-' * 50)
print(cat_val_2['Click'].value_counts(normalize=True))
print('-' * 50)
print('출산/육아')
print(cat_val_3['Click'].mean())
print('-' * 50)
print(cat_val_3['Click'].value_counts(normalize=True))
print('-' * 50)
print('패션의류')
print(cat_val_4['Click'].mean())
print('-' * 50)
print(cat_val_4['Click'].value_counts(normalize=True))
print('-' * 50)
print('생활/건강')
print(cat_val_5['Click'].mean())
print('-' * 50)
print(cat_val_5['Click'].value_counts(normalize=True))
print('-' * 50)
print('디지털/가전')
print(cat_val_6['Click'].mean())
print('-' * 50)
print(cat_val_6['Click'].value_counts(normalize=True))
print('-' * 50)
print('가구/인테리어')
print(cat_val_7['Click'].mean())
print('-' * 50)
print(cat_val_7['Click'].value_counts(normalize=True))
print('-' * 50)
print('화장품/미용')
print(cat_val_8['Click'].mean())
print('-' * 50)
print(cat_val_8['Click'].value_counts(normalize=True))
print('-' * 50)
print('패션잡화')
print(cat_val_9['Click'].mean())
print('-' * 50)
print(cat_val_9['Click'].value_counts(normalize=True))
print('-' * 50)
print('여가/생활편의/건강')
print(cat_val_10['Click'].mean())
print('-' * 50)
print(cat_val_10['Click'].value_counts(normalize=True))
print('-' * 50)

In [ ]:
#수치표현
# text = Total Data Cost 평균
bidding_drop_null['Cost'].mean()

In [ ]:
#수치표현
# text = Category1 별 Cost 평균
print('식품')
print(cat_val_1['Cost'].mean())
print('-' * 50)
print(cat_val_1['Cost'].value_counts(normalize=True))
print('-' * 50)
print('스포츠/레저')
print(cat_val_2['Cost'].mean())
print('-' * 50)
print(cat_val_2['Cost'].value_counts(normalize=True))
print('-' * 50)
print('출산/육아')
print(cat_val_3['Cost'].mean())
print('-' * 50)
print(cat_val_3['Cost'].value_counts(normalize=True))
print('-' * 50)
print('패션의류')
print(cat_val_4['Cost'].sum())
print('-' * 50)
print(cat_val_4['Cost'].value_counts(normalize=True))
print('-' * 50)
print('생활/건강')
print(cat_val_5['Cost'].mean())
print('-' * 50)
print(cat_val_5['Cost'].value_counts(normalize=True))
print('-' * 50)
print('디지털/가전')
print(cat_val_6['Cost'].mean())
print('-' * 50)
print(cat_val_6['Cost'].value_counts(normalize=True))
print('-' * 50)
print('가구/인테리어')
print(cat_val_7['Cost'].sum())
print('-' * 50)
print(cat_val_7['Cost'].value_counts(normalize=True))
print('-' * 50)
print('화장품/미용')
print(cat_val_8['Cost'].mean())
print('-' * 50)
print(cat_val_8['Cost'].value_counts(normalize=True))
print('-' * 50)
print('패션잡화')
print(cat_val_9['Cost'].mean())
print('-' * 50)
print(cat_val_9['Cost'].value_counts(normalize=True))
print('-' * 50)
print('여가/생활편의/건강')
print(cat_val_10['Cost'].mean())
print('-' * 50)
print(cat_val_10['Cost'].value_counts(normalize=True))
print('-' * 50)

In [ ]:
#수치표현
# text = Total Data Sum of AD rank 평균
bidding_drop_null['Sum of AD rank'].mean()

In [ ]:
#수치표현
#그래프표현
# text = Category1 별 Sum of AD rank 평균
print('식품')
print(cat_val_1['Sum of AD rank'].mean())
print('-' * 50)
print(cat_val_1['Sum of AD rank'].value_counts(normalize=True))
print('-' * 50)
print('스포츠/레저')
print(cat_val_2['Sum of AD rank'].mean())
print('-' * 50)
print(cat_val_2['Sum of AD rank'].value_counts(normalize=True))
print('-' * 50)
print('출산/육아')
print(cat_val_3['Sum of AD rank'].mean())
print('-' * 50)
print(cat_val_3['Sum of AD rank'].value_counts(normalize=True))
print('-' * 50)
print('패션의류')
print(cat_val_4['Sum of AD rank'].mean())
print('-' * 50)
print(cat_val_4['Sum of AD rank'].value_counts(normalize=True))
print('-' * 50)
print('생활/건강')
print(cat_val_5['Sum of AD rank'].mean())
print('-' * 50)
print(cat_val_5['Sum of AD rank'].value_counts(normalize=True))
print('-' * 50)
print('디지털/가전')
print(cat_val_6['Sum of AD rank'].mean())
print('-' * 50)
print(cat_val_6['Sum of AD rank'].value_counts(normalize=True))
print('-' * 50)
print('가구/인테리어')
print(cat_val_7['Sum of AD rank'].mean())
print('-' * 50)
print(cat_val_7['Sum of AD rank'].value_counts(normalize=True))
print('-' * 50)
print('화장품/미용')
print(cat_val_8['Sum of AD rank'].mean())
print('-' * 50)
print(cat_val_8['Sum of AD rank'].value_counts(normalize=True))
print('-' * 50)
print('패션잡화')
print(cat_val_9['Sum of AD rank'].mean())
print('-' * 50)
print(cat_val_9['Sum of AD rank'].value_counts(normalize=True))
print('-' * 50)
print('여가/생활편의/건강')
print(cat_val_10['Sum of AD rank'].mean())
print('-' * 50)
print(cat_val_10['Sum of AD rank'].value_counts(normalize=True))
print('-' * 50)

In [ ]:
#수치표현
# text = Total Data Impression 평균

bidding_drop_null['Impression'].mean()

In [ ]:
#수치표현
# text = Category1 별 Impression 평균
print('식품')
print(cat_val_1['Impression'].mean())
print('-' * 50)
print(cat_val_1['Impression'].value_counts(normalize=True))
print('-' * 50)
print('스포츠/레저')
print(cat_val_2['Impression'].mean())
print('-' * 50)
print(cat_val_2['Impression'].value_counts(normalize=True))
print('-' * 50)
print('출산/육아')
print(cat_val_3['Impression'].mean())
print('-' * 50)
print(cat_val_3['Impression'].value_counts(normalize=True))
print('-' * 50)
print('패션의류')
print(cat_val_4['Impression'].mean())
print('-' * 50)
print(cat_val_4['Impression'].value_counts(normalize=True))
print('-' * 50)
print('생활/건강')
print(cat_val_5['Impression'].mean())
print('-' * 50)
print(cat_val_5['Impression'].value_counts(normalize=True))
print('-' * 50)
print('디지털/가전')
print(cat_val_6['Impression'].mean())
print('-' * 50)
print(cat_val_6['Impression'].value_counts(normalize=True))
print('-' * 50)
print('가구/인테리어')
print(cat_val_7['Impression'].mean())
print('-' * 50)
print(cat_val_7['Impression'].value_counts(normalize=True))
print('-' * 50)
print('화장품/미용')
print(cat_val_8['Impression'].mean())
print('-' * 50)
print(cat_val_8['Impression'].value_counts(normalize=True))
print('-' * 50)
print('패션잡화')
print(cat_val_9['Impression'].mean())
print('-' * 50)
print(cat_val_9['Impression'].value_counts(normalize=True))
print('-' * 50)
print('여가/생활편의/건강')
print(cat_val_10['Impression'].mean())
print('-' * 50)
print(cat_val_10['Impression'].value_counts(normalize=True))
print('-' * 50)

In [ ]:
#수치표현
# text = Total Data Price 평균

bidding_drop_null['Price(PC)'].median()

In [ ]:
#수치표현
#그래프표현
# text = Category1 별 Price 평균
print('식품')
print(cat_val_1['Price(PC)'].median())
print('-' * 50)
print(cat_val_1['Price(PC)'].value_counts(normalize=True))
print('-' * 50)
print('스포츠/레저')
print(cat_val_2['Price(PC)'].median())
print('-' * 50)
print(cat_val_2['Price(PC)'].value_counts(normalize=True))
print('-' * 50)
print('출산/육아')
print(cat_val_3['Price(PC)'].median())
print('-' * 50)
print(cat_val_3['Price(PC)'].value_counts(normalize=True))
print('-' * 50)
print('패션의류')
print(cat_val_4['Price(PC)'].median())
print('-' * 50)
print(cat_val_4['Price(PC)'].value_counts(normalize=True))
print('-' * 50)
print('생활/건강')
print(cat_val_5['Price(PC)'].median())
print('-' * 50)
print(cat_val_5['Price(PC)'].value_counts(normalize=True))
print('-' * 50)
print('디지털/가전')
print(cat_val_6['Price(PC)'].median())
print('-' * 50)
print(cat_val_6['Price(PC)'].value_counts(normalize=True))
print('-' * 50)
print('가구/인테리어')
print(cat_val_7['Price(PC)'].median())
print('-' * 50)
print(cat_val_7['Price(PC)'].value_counts(normalize=True))
print('-' * 50)
print('화장품/미용')
print(cat_val_8['Price(PC)'].median())
print('-' * 50)
print(cat_val_8['Price(PC)'].value_counts(normalize=True))
print('-' * 50)
print('패션잡화')
print(cat_val_9['Price(PC)'].median())
print('-' * 50)
print(cat_val_9['Price(PC)'].value_counts(normalize=True))
print('-' * 50)
print('여가/생활편의/건강')
print(cat_val_10['Price(PC)'].median())
print('-' * 50)
print(cat_val_10['Price(PC)'].value_counts(normalize=True))
print('-' * 50)

## 광고별 Category 1 분석

In [ ]:
idx = (list(set(bidding_drop_null["index"].values)))
idx_cat = bidding_drop_null.drop_duplicates(['index'])
idx_cat

In [ ]:
cat_val_1 = idx_cat[idx_cat['Category Name 1'].isin(['식품'])]
cat_val_2 = idx_cat[idx_cat['Category Name 1'].isin(['스포츠/레저'])]
cat_val_3 = idx_cat[idx_cat['Category Name 1'].isin(['출산/육아'])]
cat_val_4 = idx_cat[idx_cat['Category Name 1'].isin(['패션의류'])]
cat_val_5 = idx_cat[idx_cat['Category Name 1'].isin(['생활/건강'])]
cat_val_6 = idx_cat[idx_cat['Category Name 1'].isin(['디지털/가전'])]
cat_val_7 = idx_cat[idx_cat['Category Name 1'].isin(['가구/인테리어'])]
cat_val_8 = idx_cat[idx_cat['Category Name 1'].isin(['화장품/미용'])]
cat_val_9 = idx_cat[idx_cat['Category Name 1'].isin(['패션잡화'])]
cat_val_10 = idx_cat[idx_cat['Category Name 1'].isin(['여가/생활편의'])]

In [ ]:
#수치표현
# text = 광고 Data Click 평균
idx_cat['Click'].mean()

In [ ]:
#수치표현
#그래프표현
# text = 광고 Data Category1 별 Click 평균
print('식품')
print(cat_val_1['Click'].mean())
print('-' * 50)
print(cat_val_1['Click'].value_counts(normalize=True))
print('-' * 50)
print('스포츠/레저')
print(cat_val_2['Click'].mean())
print('-' * 50)
print(cat_val_2['Click'].value_counts(normalize=True))
print('-' * 50)
print('출산/육아')
print(cat_val_3['Click'].mean())
print('-' * 50)
print(cat_val_3['Click'].value_counts(normalize=True))
print('-' * 50)
print('패션의류')
print(cat_val_4['Click'].mean())
print('-' * 50)
print(cat_val_4['Click'].value_counts(normalize=True))
print('-' * 50)
print('생활/건강')
print(cat_val_5['Click'].mean())
print('-' * 50)
print(cat_val_5['Click'].value_counts(normalize=True))
print('-' * 50)
print('디지털/가전')
print(cat_val_6['Click'].mean())
print('-' * 50)
print(cat_val_6['Click'].value_counts(normalize=True))
print('-' * 50)
print('가구/인테리어')
print(cat_val_7['Click'].mean())
print('-' * 50)
print(cat_val_7['Click'].value_counts(normalize=True))
print('-' * 50)
print('화장품/미용')
print(cat_val_8['Click'].mean())
print('-' * 50)
print(cat_val_8['Click'].value_counts(normalize=True))
print('-' * 50)
print('패션잡화')
print(cat_val_9['Click'].mean())
print('-' * 50)
print(cat_val_9['Click'].value_counts(normalize=True))
print('-' * 50)
print('여가/생활편의/건강')
print(cat_val_10['Click'].mean())
print('-' * 50)
print(cat_val_10['Click'].value_counts(normalize=True))
print('-' * 50)

In [ ]:
#수치표현
# text = 광고 Data Cost 평균

idx_cat['Cost'].mean()

In [ ]:
#수치표현
#그래프표현
# text = 광고 Data Category1 별 Cost 평균
print('식품')
print(cat_val_1['Cost'].mean())
print('-' * 50)
print(cat_val_1['Cost'].value_counts(normalize=True))
print('-' * 50)
print('스포츠/레저')
print(cat_val_2['Cost'].mean())
print('-' * 50)
print(cat_val_2['Cost'].value_counts(normalize=True))
print('-' * 50)
print('출산/육아')
print(cat_val_3['Cost'].mean())
print('-' * 50)
print(cat_val_3['Cost'].value_counts(normalize=True))
print('-' * 50)
print('패션의류')
print(cat_val_4['Cost'].mean())
print('-' * 50)
print(cat_val_4['Cost'].value_counts(normalize=True))
print('-' * 50)
print('생활/건강')
print(cat_val_5['Cost'].mean())
print('-' * 50)
print(cat_val_5['Cost'].value_counts(normalize=True))
print('-' * 50)
print('디지털/가전')
print(cat_val_6['Cost'].mean())
print('-' * 50)
print(cat_val_6['Cost'].value_counts(normalize=True))
print('-' * 50)
print('가구/인테리어')
print(cat_val_7['Cost'].mean())
print('-' * 50)
print(cat_val_7['Cost'].value_counts(normalize=True))
print('-' * 50)
print('화장품/미용')
print(cat_val_8['Cost'].mean())
print('-' * 50)
print(cat_val_8['Cost'].value_counts(normalize=True))
print('-' * 50)
print('패션잡화')
print(cat_val_9['Cost'].mean())
print('-' * 50)
print(cat_val_9['Cost'].value_counts(normalize=True))
print('-' * 50)
print('여가/생활편의/건강')
print(cat_val_10['Cost'].mean())
print('-' * 50)
print(cat_val_10['Cost'].value_counts(normalize=True))
print('-' * 50)

In [ ]:
#수치표현
# text = 광고 Data Sum of AD rank 평균
idx_cat['Sum of AD rank'].mean()

In [ ]:
#수치표현
#그래프표현
# text = 광고 Data Category1 별 Sum of AD rank 평균
print('식품')
print(cat_val_1['Sum of AD rank'].mean())
print('-' * 50)
print(cat_val_1['Sum of AD rank'].value_counts(normalize=True))
print('-' * 50)
print('스포츠/레저')
print(cat_val_2['Sum of AD rank'].mean())
print('-' * 50)
print(cat_val_2['Sum of AD rank'].value_counts(normalize=True))
print('-' * 50)
print('출산/육아')
print(cat_val_3['Sum of AD rank'].mean())
print('-' * 50)
print(cat_val_3['Sum of AD rank'].value_counts(normalize=True))
print('-' * 50)
print('패션의류')
print(cat_val_4['Sum of AD rank'].mean())
print('-' * 50)
print(cat_val_4['Sum of AD rank'].value_counts(normalize=True))
print('-' * 50)
print('생활/건강')
print(cat_val_5['Sum of AD rank'].mean())
print('-' * 50)
print(cat_val_5['Sum of AD rank'].value_counts(normalize=True))
print('-' * 50)
print('디지털/가전')
print(cat_val_6['Sum of AD rank'].mean())
print('-' * 50)
print(cat_val_6['Sum of AD rank'].value_counts(normalize=True))
print('-' * 50)
print('가구/인테리어')
print(cat_val_7['Sum of AD rank'].mean())
print('-' * 50)
print(cat_val_7['Sum of AD rank'].value_counts(normalize=True))
print('-' * 50)
print('화장품/미용')
print(cat_val_8['Sum of AD rank'].mean())
print('-' * 50)
print(cat_val_8['Sum of AD rank'].value_counts(normalize=True))
print('-' * 50)
print('패션잡화')
print(cat_val_9['Sum of AD rank'].mean())
print('-' * 50)
print(cat_val_9['Sum of AD rank'].value_counts(normalize=True))
print('-' * 50)
print('여가/생활편의/건강')
print(cat_val_10['Sum of AD rank'].mean())
print('-' * 50)
print(cat_val_10['Sum of AD rank'].value_counts(normalize=True))
print('-' * 50)

In [ ]:
#수치표현
# text = 광고 Data Impression 평균
idx_cat['Impression'].mean()

In [ ]:
#수치표현
#그래프표현
# text = 광고 Data Category1 별 Impression 평균
print('식품')
print(cat_val_1['Impression'].mean())
print('-' * 50)
print(cat_val_1['Impression'].value_counts(normalize=True))
print('-' * 50)
print('스포츠/레저')
print(cat_val_2['Impression'].mean())
print('-' * 50)
print(cat_val_2['Impression'].value_counts(normalize=True))
print('-' * 50)
print('출산/육아')
print(cat_val_3['Impression'].mean())
print('-' * 50)
print(cat_val_3['Impression'].value_counts(normalize=True))
print('-' * 50)
print('패션의류')
print(cat_val_4['Impression'].mean())
print('-' * 50)
print(cat_val_4['Impression'].value_counts(normalize=True))
print('-' * 50)
print('생활/건강')
print(cat_val_5['Impression'].mean())
print('-' * 50)
print(cat_val_5['Impression'].value_counts(normalize=True))
print('-' * 50)
print('디지털/가전')
print(cat_val_6['Impression'].mean())
print('-' * 50)
print(cat_val_6['Impression'].value_counts(normalize=True))
print('-' * 50)
print('가구/인테리어')
print(cat_val_7['Impression'].mean())
print('-' * 50)
print(cat_val_7['Impression'].value_counts(normalize=True))
print('-' * 50)
print('화장품/미용')
print(cat_val_8['Impression'].mean())
print('-' * 50)
print(cat_val_8['Impression'].value_counts(normalize=True))
print('-' * 50)
print('패션잡화')
print(cat_val_9['Impression'].mean())
print('-' * 50)
print(cat_val_9['Impression'].value_counts(normalize=True))
print('-' * 50)
print('여가/생활편의/건강')
print(cat_val_10['Impression'].mean())
print('-' * 50)
print(cat_val_10['Impression'].value_counts(normalize=True))
print('-' * 50)

In [ ]:
#수치표현
# text = 광고 Data Price 평균

idx_cat['Price(PC)'].median()

In [ ]:
#수치표현
#그래프표현
# text = 광고 Data Category1 별 Price 평균
print('식품')
print(cat_val_1['Price(PC)'].median())
print('-' * 50)
print(cat_val_1['Price(PC)'].value_counts(normalize=True))
print('-' * 50)
print('스포츠/레저')
print(cat_val_2['Price(PC)'].median())
print('-' * 50)
print(cat_val_2['Price(PC)'].value_counts(normalize=True))
print('-' * 50)
print('출산/육아')
print(cat_val_3['Price(PC)'].median())
print('-' * 50)
print(cat_val_3['Price(PC)'].value_counts(normalize=True))
print('-' * 50)
print('패션의류')
print(cat_val_4['Price(PC)'].median())
print('-' * 50)
print(cat_val_4['Price(PC)'].value_counts(normalize=True))
print('-' * 50)
print('생활/건강')
print(cat_val_5['Price(PC)'].median())
print('-' * 50)
print(cat_val_5['Price(PC)'].value_counts(normalize=True))
print('-' * 50)
print('디지털/가전')
print(cat_val_6['Price(PC)'].median())
print('-' * 50)
print(cat_val_6['Price(PC)'].value_counts(normalize=True))
print('-' * 50)
print('가구/인테리어')
print(cat_val_7['Price(PC)'].median())
print('-' * 50)
print(cat_val_7['Price(PC)'].value_counts(normalize=True))
print('-' * 50)
print('화장품/미용')
print(cat_val_8['Price(PC)'].median())
print('-' * 50)
print(cat_val_8['Price(PC)'].value_counts(normalize=True))
print('-' * 50)
print('패션잡화')
print(cat_val_9['Price(PC)'].median())
print('-' * 50)
print(cat_val_9['Price(PC)'].value_counts(normalize=True))
print('-' * 50)
print('여가/생활편의/건강')
print(cat_val_10['Price(PC)'].median())
print('-' * 50)
print(cat_val_10['Price(PC)'].value_counts(normalize=True))
print('-' * 50)

# Prophet

In [ ]:
bidding_drop_null

In [ ]:
bidding_drop_null["Year"] = bidding_drop_null.index.year 
bidding_drop_null["Quarter"] = bidding_drop_null.index.quarter
bidding_drop_null["Month"] = bidding_drop_null.index.month
bidding_drop_null["Week"] = bidding_drop_null.index.week
bidding_drop_null["Weekday"] = bidding_drop_null.index.weekday
bidding_drop_null["Day"] = bidding_drop_null.index.day
bidding_drop_null["Dayofyear"] = bidding_drop_null.index.dayofyear
bidding_drop_null["Date"] = pd.DatetimeIndex(bidding_drop_null.index).date

In [ ]:
date_list = list(set(bidding_drop_null['Date']))
columns = ['ds', 'y']                 
bid_click_mean = pd.DataFrame(columns=columns)
for date in date_list:
    bid_mean = bidding_drop_null.loc[bidding_drop_null['Date'] == date]
    bid_click_mean = bid_click_mean.append(pd.DataFrame([[date, bid_mean['Click'].mean()]], columns=columns), ignore_index=True)
bid_click_mean

In [ ]:
date_list = list(set(bidding_drop_null['Date']))
columns = ['ds', 'y']                 
bid_cost_mean = pd.DataFrame(columns=columns)
for date in date_list:
    bid_mean = bidding_drop_null.loc[bidding_drop_null['Date'] == date]
    bid_cost_mean = bid_cost_mean.append(pd.DataFrame([[date, bid_mean['Cost'].mean()]], columns=columns), ignore_index=True)
bid_cost_mean

In [ ]:
date_list = list(set(bidding_drop_null['Date']))
columns = ['ds', 'y']                 
bid_imp_mean = pd.DataFrame(columns=columns)
for date in date_list:
    bid_mean = bidding_drop_null.loc[bidding_drop_null['Date'] == date]
    bid_imp_mean = bid_imp_mean.append(pd.DataFrame([[date, bid_mean['Impression'].mean()]], columns=columns), ignore_index=True)
bid_imp_mean

In [ ]:
prophet = Prophet()
prophet.fit(bid_click_mean)


# create a future data frame 
future = prophet.make_future_dataframe(periods=7, freq = 'D')
forecast = prophet.predict(future)

fig2 = prophet.plot_components(forecast)
plt.show()

In [ ]:
prophet = Prophet()
prophet.fit(bid_cost_mean)


# create a future data frame 
future = prophet.make_future_dataframe(periods=7, freq = 'D')
forecast = prophet.predict(future)

fig2 = prophet.plot_components(forecast)
plt.show()

In [ ]:
prophet = Prophet()
prophet.fit(bid_imp_mean)


# create a future data frame 
future = prophet.make_future_dataframe(periods=7, freq = 'D')
forecast = prophet.predict(future)

fig2 = prophet.plot_components(forecast)
plt.show()

# Categocy1 평가모델(NDCG, MAP)

In [ ]:
def NDCG(selected,ordered):
    import numpy as np
    from tqdm import tqdm
    DCG = 0
    iDCG = 0
    
    selected_result = ((np.array(selected) - 1)==0)
    for idx, (i,k) in tqdm(enumerate(zip(selected_result,ordered))):
        if i==True:
            DCG += ((max(ordered) - (k-1)) / (np.log2(idx+1 + 1)))
            
    cum_index = 0
    for idx, i in tqdm(enumerate(range(sum(selected)))):
        iDCG += ((len(selected) - cum_index) / (np.log2(idx+1 + 1)))
        cum_index += 1
    
    NDCG = DCG / iDCG
    
    return NDCG

In [ ]:
bidding_drop_null[bidding_drop_null['Month'] == 1]

In [ ]:
idx_bid = bidding_drop_null.drop_duplicates(['index'])

In [ ]:
idx_bid[idx_bid['Month'] == 1]

In [ ]:
month_10 = idx_bid[idx_bid['Month'].isin([10])]
month_11 = idx_bid[idx_bid['Month'].isin([11])]
month_12 = idx_bid[idx_bid['Month'].isin([12])]
month_1 = idx_bid[idx_bid['Month'].isin([1])]

In [ ]:
cat_val_1 = idx_bid[idx_bid['Category Name 1'].isin(['식품'])]
cat_val_2 = idx_bid[idx_bid['Category Name 1'].isin(['스포츠/레저'])]
cat_val_3 = idx_bid[idx_bid['Category Name 1'].isin(['출산/육아'])]
cat_val_4 = idx_bid[idx_bid['Category Name 1'].isin(['패션의류'])]
cat_val_5 = idx_bid[idx_bid['Category Name 1'].isin(['생활/건강'])]
cat_val_6 = idx_bid[idx_bid['Category Name 1'].isin(['디지털/가전'])]
cat_val_7 = idx_bid[idx_bid['Category Name 1'].isin(['가구/인테리어'])]
cat_val_8 = idx_bid[idx_bid['Category Name 1'].isin(['화장품/미용'])]
cat_val_9 = idx_bid[idx_bid['Category Name 1'].isin(['패션잡화'])]
cat_val_10 = idx_bid[idx_bid['Category Name 1'].isin(['여가/생활편의'])]

In [ ]:
cat_val_1_11 = cat_val_1[cat_val_1['Month'].isin([11])]
cat_val_1_12 = cat_val_1[cat_val_1['Month'].isin([12])]
cat_val_1_1 = cat_val_1[cat_val_1['Month'].isin([1])]
cat_val_1_10 = cat_val_1[cat_val_1['Month'].isin([10])]

In [ ]:
cat_val_2_11 = cat_val_2[cat_val_2['Month'].isin([11])]
cat_val_2_12 = cat_val_2[cat_val_2['Month'].isin([12])]
cat_val_2_1 = cat_val_2[cat_val_2['Month'].isin([1])]
cat_val_2_10 = cat_val_2[cat_val_2['Month'].isin([10])]

In [ ]:
cat_val_3_11 = cat_val_3[cat_val_3['Month'].isin([11])]
cat_val_3_12 = cat_val_3[cat_val_3['Month'].isin([12])]
cat_val_3_1 = cat_val_3[cat_val_3['Month'].isin([1])]
cat_val_3_10 = cat_val_3[cat_val_3['Month'].isin([10])]

In [ ]:
cat_val_4_11 = cat_val_4[cat_val_4['Month'].isin([11])]
cat_val_4_12 = cat_val_4[cat_val_4['Month'].isin([12])]
cat_val_4_1 = cat_val_4[cat_val_4['Month'].isin([1])]
cat_val_4_10 = cat_val_4[cat_val_4['Month'].isin([10])]

In [ ]:
cat_val_5_11 = cat_val_5[cat_val_5['Month'].isin([11])]
cat_val_5_12 = cat_val_5[cat_val_5['Month'].isin([12])]
cat_val_5_1 = cat_val_5[cat_val_5['Month'].isin([1])]
cat_val_5_10 = cat_val_5[cat_val_5['Month'].isin([10])]

In [ ]:
cat_val_6_11 = cat_val_6[cat_val_6['Month'].isin([11])]
cat_val_6_12 = cat_val_6[cat_val_6['Month'].isin([12])]
cat_val_6_1 = cat_val_6[cat_val_6['Month'].isin([1])]
cat_val_6_10 = cat_val_6[cat_val_6['Month'].isin([10])]

In [ ]:
cat_val_7_11 = cat_val_7[cat_val_7['Month'].isin([11])]
cat_val_7_12 = cat_val_7[cat_val_7['Month'].isin([12])]
cat_val_7_1 = cat_val_7[cat_val_7['Month'].isin([1])]
cat_val_7_10 = cat_val_7[cat_val_7['Month'].isin([10])]

In [ ]:
cat_val_8_11 = cat_val_8[cat_val_8['Month'].isin([11])]
cat_val_8_12 = cat_val_8[cat_val_8['Month'].isin([12])]
cat_val_8_1 = cat_val_8[cat_val_8['Month'].isin([1])]
cat_val_8_10 = cat_val_8[cat_val_8['Month'].isin([10])]

In [ ]:
cat_val_9_11 = cat_val_9[cat_val_9['Month'].isin([11])]
cat_val_9_12 = cat_val_9[cat_val_9['Month'].isin([12])]
cat_val_9_1 = cat_val_9[cat_val_9['Month'].isin([1])]
cat_val_9_10 = cat_val_9[cat_val_9['Month'].isin([10])]

In [ ]:
cat_val_10_11 = cat_val_10[cat_val_10['Month'].isin([11])]
cat_val_10_12 = cat_val_10[cat_val_10['Month'].isin([12])]
cat_val_10_1 = cat_val_10[cat_val_10['Month'].isin([1])]
cat_val_10_10 = cat_val_10[cat_val_10['Month'].isin([10])]

# 대시보드 정리

## 상품가격별 분석
#결론 : 미미하지만 가설성립, 5~10만원대 상품들의 click과 cost의 상관관계가 낮아졌다 개선이필요하다

In [ ]:
# 수치표현
# 그래프표현
# text = 5만원 이하 상품 가격 데이터 특징
price_Analysis(price_down_5)

In [ ]:
# 수치표현
# 그래프표현
# text = 5~10만원 상품 가격 데이터 특징
price_Analysis(price_5_10)

In [ ]:
# 수치표현
# 그래프표현
# text = 10~15만원 상품 가격 데이터 특징
price_Analysis(price_10_15)

In [ ]:
# 수치표현
# 그래프표현
# text = 15~20만원 상품 가격 데이터 특징
price_Analysis(price_15_20)

In [ ]:
# 수치표현
# 그래프표현
# text = 20만원 이상 상품 가격 데이터 특징
price_Analysis(price_up_20)

In [ ]:
# 그래프 표현
# text = 5만원 이하 상품 가격 데이터 상관관계
corr_heat(price_down_5)

In [ ]:
# 그래프 표현
# text = 5~10만원 상품 가격 데이터 상관관계
corr_heat(price_5_10)

In [ ]:
# 그래프 표현
# text = 10~15만원 상품 가격 데이터 상관관계
corr_heat(price_10_15)

In [ ]:
# 그래프 표현
# text = 15~20만원 상품 가격 데이터 상관관계
corr_heat(price_15_20)

In [ ]:
# 그래프 표현
# text = 20만원 이상 상품 가격 데이터 상관관계
corr_heat(price_up_20)

## Rank 구간별 분석
#결론 : 상식적인 결과의 연속, 특이점은 상품가격이 1등에 가까울수록 높아진다

In [ ]:
#수치표현
# text = Rank 구간별 광고 비율
print("rank_down_26 광고숫자비율")
print((len(list(set(rank_down_26['index'].values)))))
print('-' * 50)
print("rank_26_46 광고숫자비율")
print((len(list(set(rank_26_46['index'].values)))))
print('-' * 50)
print("rank_46_66 광고숫자비율")
print((len(list(set(rank_46_66['index'].values)))))
print('-' * 50)
print("rank_66_86 광고숫자비율")
print((len(list(set(rank_66_86['index'].values)))))
print('-' * 50)
print("rank_up_86 광고숫자비율")
print((len(list(set(rank_up_86['index'].values)))))
print('-' * 50)

In [ ]:
#수치표현
# text = Rank 구간별 Click 평균
print('rank_down_26')
print(rank_down_26['Click'].mean())
print('-' * 50)
print(rank_down_26['Click'].value_counts(normalize=True))
print('-' * 50)
print('rank_26_46')
print(rank_26_46['Click'].mean())
print('-' * 50)
print(rank_26_46['Click'].value_counts(normalize=True))
print('-' * 50)
print('rank_46_66')
print(rank_46_66['Click'].mean())
print('-' * 50)
print(rank_46_66['Click'].value_counts(normalize=True))
print('-' * 50)
print('rank_66_86')
print(rank_66_86['Click'].mean())
print('-' * 50)
print(rank_66_86['Click'].value_counts(normalize=True))
print('-' * 50)
print('rank_up_86')
print(rank_up_86['Click'].mean())
print('-' * 50)
print(rank_up_86['Click'].value_counts(normalize=True))
print('-' * 50)

In [ ]:
#수치표현
# text = Rank 구간별 Cost 평균
print('rank_down_26')
print(rank_down_26['Cost'].mean())
print('-' * 50)
print(rank_down_26['Cost'].value_counts(normalize=True))
print('-' * 50)
print('rank_26_46')
print(rank_26_46['Cost'].mean())
print('-' * 50)
print(rank_26_46['Cost'].value_counts(normalize=True))
print('-' * 50)
print('rank_46_66')
print(rank_46_66['Cost'].mean())
print('-' * 50)
print(rank_46_66['Cost'].value_counts(normalize=True))
print('-' * 50)
print('rank_66_86')
print(rank_66_86['Cost'].mean())
print('-' * 50)
print(rank_66_86['Cost'].value_counts(normalize=True))
print('-' * 50)
print('rank_up_86')
print(rank_up_86['Cost'].mean())
print('-' * 50)
print(rank_up_86['Cost'].value_counts(normalize=True))
print('-' * 50)

In [ ]:
#수치표현
# text = Rank 구간별 Impression 평균
print('rank_down_26')
print(rank_down_26['Impression'].mean())
print('-' * 50)
print(rank_down_26['Impression'].value_counts(normalize=True))
print('-' * 50)
print('rank_26_46')
print(rank_26_46['Impression'].mean())
print('-' * 50)
print(rank_26_46['Impression'].value_counts(normalize=True))
print('-' * 50)
print('rank_46_66')
print(rank_46_66['Impression'].mean())
print('-' * 50)
print(rank_46_66['Impression'].value_counts(normalize=True))
print('-' * 50)
print('rank_66_86')
print(rank_66_86['Impression'].mean())
print('-' * 50)
print(rank_66_86['Impression'].value_counts(normalize=True))
print('-' * 50)
print('rank_up_86')
print(rank_up_86['Impression'].mean())
print('-' * 50)
print(rank_up_86['Impression'].value_counts(normalize=True))
print('-' * 50)

In [ ]:
# 수치표현
# text = Rank 구간별 Price(PC) 중간값
print('rank_down_26')
print(rank_down_26['Price(PC)'].median())
print('-' * 50)
print(rank_down_26['Price(PC)'].value_counts(normalize=True))
print('-' * 50)
print('rank_26_46')
print(rank_26_46['Price(PC)'].median())
print('-' * 50)
print(rank_26_46['Price(PC)'].value_counts(normalize=True))
print('-' * 50)
print('rank_46_66')
print(rank_46_66['Price(PC)'].median())
print('-' * 50)
print(rank_46_66['Price(PC)'].value_counts(normalize=True))
print('-' * 50)
print('rank_66_86')
print(rank_66_86['Price(PC)'].median())
print('-' * 50)
print(rank_66_86['Price(PC)'].value_counts(normalize=True))
print('-' * 50)
print('rank_up_86')
print(rank_up_86['Price(PC)'].median())
print('-' * 50)
print(rank_up_86['Price(PC)'].value_counts(normalize=True))
print('-' * 50)

## 카테고리별 특징
#결론 : 시계열 모델을 돌려보니 트랜드가 점차 낮아지는 특징을 발견하여 월별 NDCG를 적용
        돌려보니 점점 좋아지는 카테고리가 보임, 서브 카테고리로써 좋은 시도를 해본다면 시너지가 나오지 않을까?
        
#특이점 : 식품은 광고가 없어서 0, 주석처리한건 0값이 뜬다, 1월은 595개 밖에 없어서 생략, ccc는 임의로 먹인 이름

In [ ]:
# 수치표현
# text = 시계열 모델로 Click 특징찾기

prophet = Prophet()
prophet.fit(bid_click_mean)


# create a future data frame 
future = prophet.make_future_dataframe(periods=7, freq = 'D')
forecast = prophet.predict(future)

fig2 = prophet.plot_components(forecast)
plt.show()

In [ ]:
# 수치표현
# text = 시계열 모델로 Impression 특징찾기

prophet = Prophet()
prophet.fit(bid_imp_mean)


# create a future data frame 
future = prophet.make_future_dataframe(periods=7, freq = 'D')
forecast = prophet.predict(future)

fig2 = prophet.plot_components(forecast)
plt.show()

In [ ]:
# 수치표현
# text = 시계열 모델로 Cost 특징찾기

prophet = Prophet()
prophet.fit(bid_cost_mean)


# create a future data frame 
future = prophet.make_future_dataframe(periods=7, freq = 'D')
forecast = prophet.predict(future)

fig2 = prophet.plot_components(forecast)
plt.show()

In [ ]:
# 수치표현
# 그래프표현
# text = 월별 NDCG 평가
print(f'10월 Total [Click] : {(month_10["Click"].mean())}')
print(f'11월 Total [Click] : {(month_11["Click"].mean())}')
print(f'12월 Total [Click] : {(month_12["Click"].mean())}')
# print(f'1월 Total [Click] : {(month_1["Click"].mean())}')

print('-'*50)

print(f'10월 Total [Cost] : {(month_10["Cost"].mean())}')
print(f'11월 Total [Cost] : {(month_11["Cost"].mean())}')
print(f'12월 Total [Cost] : {(month_12["Cost"].mean())}')
# print(f'1월 Total [Cost] : {(month_1["Cost"].mean())}')

print('-'*50)

print(f'10월 Total [Impression] : {(month_10["Impression"].mean())}')
print(f'11월 Total [Impression] : {(month_11["Impression"].mean())}')
print(f'12월 Total [Impression] : {(month_12["Impression"].mean())}')
# print(f'1월 Total [Impression] : {(month_1["Impression"].mean())}')

print('-'*50)

print(f'10월 Total [Sum of AD rank] : {(month_10["Sum of AD rank"].mean())}')
print(f'11월 Total [Sum of AD rank] : {(month_11["Sum of AD rank"].mean())}')
print(f'12월 Total [Sum of AD rank] : {(month_12["Sum of AD rank"].mean())}')
# print(f'1월 Total [Sum of AD rank] : {(month_1["Sum of AD rank"].mean())}')

print('-'*50)


print(f'10월 Total [CCC] : {(month_10["Click"].mean()) / (month_10["Cost"].mean())}')
print(f'11월 Total [CCC] : {(month_11["Click"].mean()) / (month_11["Cost"].mean())}')
print(f'12월 Total [CCC] : {(month_12["Click"].mean()) / (month_12["Cost"].mean())}')

In [ ]:
# # 수치표현
# # 그래프표현
# # text = 월별 패션의류
# print(f'식품 10월 [Click] = {cat_val_1_10["Click"].mean()}')
# print(f'식품 11월 [Click] = {cat_val_1_11["Click"].mean()}')
# print(f'식품 12월 [Click] = {cat_val_1_12["Click"].mean()}')
# print(f'식품 1월 [Click] = {cat_val_1_1["Click"].mean()}')

# print('-'*50)

# print(f'식품 10월 [Cost] = {cat_val_1_10["Cost"].mean()}')
# print(f'식품 11월 [Cost ] = {cat_val_1_11["Cost"].mean()}')
# print(f'식품 12월 [Cost] = {cat_val_1_12["Cost"].mean()}')
# print(f'식품 1월 [Cost] = {cat_val_1_1["Cost"].mean()}')

# print('-'*50)

# print(f'식품 10월 [Impression] = {cat_val_1_10["Impression"].mean()}')
# print(f'식품 11월 [Impression] = {cat_val_1_11["Impression"].mean()}')
# print(f'식품 12월 [Impression] = {cat_val_1_12["Impression"].mean()}')
# print(f'식품 1월 [Impression] = {cat_val_1_1["Impression"].mean()}')

# print('-'*50)

# print(f'식품 10월 [Sum of AD rank] = {cat_val_1_10["Sum of AD rank"].mean()}')
# print(f'식품 11월 [Sum of AD rank] = {cat_val_1_11["Sum of AD rank"].mean()}')
# print(f'식품 12월 [Sum of AD rank] = {cat_val_1_12["Sum of AD rank"].mean()}')
# print(f'식품 1월 [Sum of AD rank] = {cat_val_1_1["Sum of AD rank"].mean()}')

# print('-'*50)

# print(f'10월 식품 [CCC] : {(cat_val_1_10["Click"].mean()) / (cat_val_1_10["Cost"].mean())}')
# print(f'12월 식품 [CCC] : {(cat_val_1_11["Click"].mean()) / (cat_val_1_11["Cost"].mean())}')
# print(f'12월 식품 [CCC] : {(cat_val_1_12["Click"].mean()) / (cat_val_1_12["Cost"].mean())}')

In [ ]:
# 수치표현
# 그래프표현
# text = 월별 스포츠/레저
print(f'스포츠/레저 10월 [Click] = {cat_val_2_10["Click"].mean()}')
print(f'스포츠/레저 11월 [Click] = {cat_val_2_11["Click"].mean()}')
print(f'스포츠/레저 12월 [Click] = {cat_val_2_12["Click"].mean()}')
# print(f'스포츠/레저 1월 [Click] = {cat_val_2_1["Click"].mean()}')

print('-'*50)

print(f'스포츠/레저 10월 [Cost] = {cat_val_2_10["Cost"].mean()}')
print(f'스포츠/레저 11월 [Cost ] = {cat_val_2_11["Cost"].mean()}')
print(f'스포츠/레저 12월 [Cost] = {cat_val_2_12["Cost"].mean()}')
# print(f'스포츠/레저 1월 [Cost] = {cat_val_2_1["Cost"].mean()}')

print('-'*50)

print(f'스포츠/레저 10월 [Impression] = {cat_val_2_10["Impression"].mean()}')
print(f'스포츠/레저 11월 [Impression] = {cat_val_2_11["Impression"].mean()}')
print(f'스포츠/레저 12월 [Impression] = {cat_val_2_12["Impression"].mean()}')
# print(f'스포츠/레저 1월 [Impression] = {cat_val_2_1["Impression"].mean()}')

print('-'*50)

print(f'스포츠/레저 10월 [Sum of AD rank] = {cat_val_2_10["Sum of AD rank"].mean()}')
print(f'스포츠/레저 11월 [Sum of AD rank] = {cat_val_2_11["Sum of AD rank"].mean()}')
print(f'스포츠/레저 12월 [Sum of AD rank] = {cat_val_2_12["Sum of AD rank"].mean()}')
# print(f'스포츠/레저 1월 [Sum of AD rank] = {cat_val_2_1["Sum of AD rank"].mean()}')


print('-'*50)

print(f'10월 스포츠/레저 [CCC] : {(cat_val_2_10["Click"].mean()) / (cat_val_2_10["Cost"].mean())}')
print(f'11월 스포츠/레저 [CCC] : {(cat_val_2_11["Click"].mean()) / (cat_val_2_11["Cost"].mean())}')
print(f'12월 스포츠/레저 [CCC] : {(cat_val_2_12["Click"].mean()) / (cat_val_2_12["Cost"].mean())}')

In [ ]:
# 수치표현
# 그래프표현
# text = 월별 출산/육아
print(f'출산/육아 10월 [Click] = {cat_val_3_10["Click"].mean()}')
print(f'출산/육아 11월 [Click] = {cat_val_3_11["Click"].mean()}')
print(f'출산/육아 12월 [Click] = {cat_val_3_12["Click"].mean()}')
# print(f'출산/육아 1월 [Click] = {cat_val_3_1["Click"].mean()}')

print('-'*50)

print(f'출산/육아 10월 [Cost] = {cat_val_3_10["Cost"].mean()}')
print(f'출산/육아 11월 [Cost ] = {cat_val_3_11["Cost"].mean()}')
print(f'출산/육아 12월 [Cost] = {cat_val_3_12["Cost"].mean()}')
# print(f'출산/육아 1월 [Cost] = {cat_val_3_1["Cost"].mean()}')

print('-'*50)

print(f'출산/육아 10월 [Impression] = {cat_val_3_10["Impression"].mean()}')
print(f'출산/육아 11월 [Impression] = {cat_val_3_11["Impression"].mean()}')
print(f'출산/육아 12월 [Impression] = {cat_val_3_12["Impression"].mean()}')
# print(f'출산/육아 1월 [Impression] = {cat_val_3_1["Impression"].mean()}')

print('-'*50)

print(f'출산/육아 10월 [Sum of AD rank] = {cat_val_3_10["Sum of AD rank"].mean()}')
print(f'출산/육아 11월 [Sum of AD rank] = {cat_val_3_11["Sum of AD rank"].mean()}')
print(f'출산/육아 12월 [Sum of AD rank] = {cat_val_3_12["Sum of AD rank"].mean()}')
# print(f'출산/육아 1월 [Sum of AD rank] = {cat_val_3_1["Sum of AD rank"].mean()}')

print('-'*50)

print(f'10월 출산/육아 [CCC] : {(cat_val_3_10["Click"].mean()) / (cat_val_3_10["Cost"].mean())}')
print(f'12월 출산/육아 [CCC] : {(cat_val_3_11["Click"].mean()) / (cat_val_3_11["Cost"].mean())}')
print(f'12월 출산/육아 [CCC] : {(cat_val_3_12["Click"].mean()) / (cat_val_3_12["Cost"].mean())}')

In [ ]:
# 수치표현
# 그래프표현
# text = 월별 패션의류
print(f'패션의류 10월 [Click] = {cat_val_4_10["Click"].mean()}')
print(f'패션의류 11월 [Click] = {cat_val_4_11["Click"].mean()}')
print(f'패션의류 12월 [Click] = {cat_val_4_12["Click"].mean()}')
# print(f'패션의류 1월 [Click] = {cat_val_4_1["Click"].mean()}')

print('-'*50)

print(f'패션의류 10월 [Cost] = {cat_val_4_10["Cost"].mean()}')
print(f'패션의류 11월 [Cost ] = {cat_val_4_11["Cost"].mean()}')
print(f'패션의류 12월 [Cost] = {cat_val_4_12["Cost"].mean()}')
# print(f'패션의류 1월 [Cost] = {cat_val_4_1["Cost"].mean()}')

print('-'*50)

print(f'패션의류 10월 [Impression] = {cat_val_4_10["Impression"].mean()}')
print(f'패션의류 11월 [Impression] = {cat_val_4_11["Impression"].mean()}')
print(f'패션의류 12월 [Impression] = {cat_val_4_12["Impression"].mean()}')
# print(f'패션의류 1월 [Impression] = {cat_val_4_1["Impression"].mean()}')

print('-'*50)

print(f'패션의류 10월 [Sum of AD rank] = {cat_val_4_10["Sum of AD rank"].mean()}')
print(f'패션의류 11월 [Sum of AD rank] = {cat_val_4_11["Sum of AD rank"].mean()}')
print(f'패션의류 12월 [Sum of AD rank] = {cat_val_4_12["Sum of AD rank"].mean()}')
# print(f'패션의류 1월 [Sum of AD rank] = {cat_val_4_1["Sum of AD rank"].mean()}')

print('-'*50)

print(f'10월 패션의류 [CCC] : {(cat_val_4_10["Click"].mean()) / (cat_val_4_10["Cost"].mean())}')
print(f'11월 패션의류 [CCC] : {(cat_val_4_11["Click"].mean()) / (cat_val_4_11["Cost"].mean())}')
print(f'12월 패션의류 [CCC] : {(cat_val_4_12["Click"].mean()) / (cat_val_4_12["Cost"].mean())}')

In [ ]:

# 수치표현
# 그래프표현
# text = 월별 생활/건강
print(f'생활/건강 10월 [Click] = {cat_val_5_10["Click"].mean()}')
print(f'생활/건강 11월 [Click] = {cat_val_5_11["Click"].mean()}')
print(f'생활/건강 12월 [Click] = {cat_val_5_12["Click"].mean()}')
# print(f'생활/건강 1월 [Click] = {cat_val_5_1["Click"].mean()}')

print('-'*50)

print(f'생활/건강 10월 [Cost] = {cat_val_5_10["Cost"].mean()}')
print(f'생활/건강 11월 [Cost ] = {cat_val_5_11["Cost"].mean()}')
print(f'생활/건강 12월 [Cost] = {cat_val_5_12["Cost"].mean()}')
# print(f'생활/건강 1월 [Cost] = {cat_val_5_1["Cost"].mean()}')

print('-'*50)

print(f'생활/건강 10월 [Impression] = {cat_val_5_10["Impression"].mean()}')
print(f'생활/건강 11월 [Impression] = {cat_val_5_11["Impression"].mean()}')
print(f'생활/건강 12월 [Impression] = {cat_val_5_12["Impression"].mean()}')
# print(f'생활/건강 1월 [Impression] = {cat_val_5_1["Impression"].mean()}')

print('-'*50)

print(f'생활/건강 10월 [Sum of AD rank] = {cat_val_5_10["Sum of AD rank"].mean()}')
print(f'생활/건강 11월 [Sum of AD rank] = {cat_val_5_11["Sum of AD rank"].mean()}')
print(f'생활/건강 12월 [Sum of AD rank] = {cat_val_5_12["Sum of AD rank"].mean()}')
# print(f'생활/건강 1월 [Sum of AD rank] = {cat_val_5_1["Sum of AD rank"].mean()}')

print('-'*50)

print(f'10월 생활/건강 [CCC] : {(cat_val_5_10["Click"].mean()) / (cat_val_5_10["Cost"].mean())}')
print(f'11월 생활/건강 [CCC] : {(cat_val_5_11["Click"].mean()) / (cat_val_5_11["Cost"].mean())}')
print(f'12월 생활/건강 [CCC] : {(cat_val_5_12["Click"].mean()) / (cat_val_5_12["Cost"].mean())}')

In [ ]:
# 수치표현
# 그래프표현
# text = 월별 디지털/가전
print(f'디지털/가전 10월 [Click] = {cat_val_6_10["Click"].mean()}')
print(f'디지털/가전 11월 [Click] = {cat_val_6_11["Click"].mean()}')
print(f'디지털/가전 12월 [Click] = {cat_val_6_12["Click"].mean()}')
print(f'디지털/가전 1월 [Click] = {cat_val_6_1["Click"].mean()}')

print('-'*50)

print(f'디지털/가전 10월 [Cost] = {cat_val_6_10["Cost"].mean()}')
print(f'디지털/가전 11월 [Cost ] = {cat_val_6_11["Cost"].mean()}')
print(f'디지털/가전 12월 [Cost] = {cat_val_6_12["Cost"].mean()}')
print(f'디지털/가전 1월 [Cost] = {cat_val_6_1["Cost"].mean()}')

print('-'*50)

print(f'디지털/가전 10월 [Impression] = {cat_val_6_10["Impression"].mean()}')
print(f'디지털/가전 11월 [Impression] = {cat_val_6_11["Impression"].mean()}')
print(f'디지털/가전 12월 [Impression] = {cat_val_6_12["Impression"].mean()}')
print(f'디지털/가전 1월 [Impression] = {cat_val_6_1["Impression"].mean()}')

print('-'*50)

print(f'디지털/가전 10월 [Sum of AD rank] = {cat_val_6_10["Sum of AD rank"].mean()}')
print(f'디지털/가전 11월 [Sum of AD rank] = {cat_val_6_11["Sum of AD rank"].mean()}')
print(f'디지털/가전 12월 [Sum of AD rank] = {cat_val_6_12["Sum of AD rank"].mean()}')
print(f'디지털/가전 1월 [Sum of AD rank] = {cat_val_6_1["Sum of AD rank"].mean()}')

print('-'*50)

print(f'10월 디지털/가전 [CCC] : {(cat_val_6_10["Click"].mean()) / (cat_val_6_10["Cost"].mean())}')
print(f'12월 디지털/가전 [CCC] : {(cat_val_6_11["Click"].mean()) / (cat_val_6_11["Cost"].mean())}')
print(f'12월 디지털/가전 [CCC] : {(cat_val_6_12["Click"].mean()) / (cat_val_6_12["Cost"].mean())}')

In [ ]:
# 수치표현
# 그래프표현
# text = 월별 가구/인테리어
print(f'가구/인테리어 10월 [Click] = {cat_val_7_10["Click"].mean()}')
print(f'가구/인테리어 11월 [Click] = {cat_val_7_11["Click"].mean()}')
print(f'가구/인테리어 12월 [Click] = {cat_val_7_12["Click"].mean()}')
print(f'가구/인테리어 1월 [Click] = {cat_val_7_1["Click"].mean()}')

print('-'*50)

print(f'가구/인테리어 10월 [Cost] = {cat_val_7_10["Cost"].mean()}')
print(f'가구/인테리어 11월 [Cost ] = {cat_val_7_11["Cost"].mean()}')
print(f'가구/인테리어 12월 [Cost] = {cat_val_7_12["Cost"].mean()}')
print(f'가구/인테리어 1월 [Cost] = {cat_val_7_1["Cost"].mean()}')

print('-'*50)

print(f'가구/인테리어 10월 [Impression] = {cat_val_7_10["Impression"].mean()}')
print(f'가구/인테리어 11월 [Impression] = {cat_val_7_11["Impression"].mean()}')
print(f'가구/인테리어 12월 [Impression] = {cat_val_7_12["Impression"].mean()}')
print(f'가구/인테리어 1월 [Impression] = {cat_val_7_1["Impression"].mean()}')

print('-'*50)

print(f'가구/인테리어 10월 [Sum of AD rank] = {cat_val_7_10["Sum of AD rank"].mean()}')
print(f'가구/인테리어 11월 [Sum of AD rank] = {cat_val_7_11["Sum of AD rank"].mean()}')
print(f'가구/인테리어 12월 [Sum of AD rank] = {cat_val_7_12["Sum of AD rank"].mean()}')
print(f'가구/인테리어 1월 [Sum of AD rank] = {cat_val_7_1["Sum of AD rank"].mean()}')

print('-'*50)

print(f'10월 가구/인테리어 [CCC] : {(cat_val_7_10["Click"].mean()) / (cat_val_7_10["Cost"].mean())}')
print(f'12월 가구/인테리어 [CCC] : {(cat_val_7_11["Click"].mean()) / (cat_val_7_11["Cost"].mean())}')
print(f'12월 가구/인테리어 [CCC] : {(cat_val_7_12["Click"].mean()) / (cat_val_7_12["Cost"].mean())}')

In [ ]:
# 수치표현
# 그래프표현
# text = 월별 화장품/미용
print(f'화장품/미용 10월 [Click] = {cat_val_8_10["Click"].mean()}')
print(f'화장품/미용 11월 [Click] = {cat_val_8_11["Click"].mean()}')
print(f'화장품/미용 12월 [Click] = {cat_val_8_12["Click"].mean()}')
print(f'화장품/미용 1월 [Click] = {cat_val_8_1["Click"].mean()}')

print('-'*50)

print(f'화장품/미용 10월 [Cost] = {cat_val_8_10["Cost"].mean()}')
print(f'화장품/미용 11월 [Cost ] = {cat_val_8_11["Cost"].mean()}')
print(f'화장품/미용 12월 [Cost] = {cat_val_8_12["Cost"].mean()}')
print(f'화장품/미용 1월 [Cost] = {cat_val_8_1["Cost"].mean()}')

print('-'*50)

print(f'화장품/미용 10월 [Impression] = {cat_val_8_10["Impression"].mean()}')
print(f'화장품/미용 11월 [Impression] = {cat_val_8_11["Impression"].mean()}')
print(f'화장품/미용 12월 [Impression] = {cat_val_8_12["Impression"].mean()}')
print(f'화장품/미용 1월 [Impression] = {cat_val_8_1["Impression"].mean()}')

print('-'*50)

print(f'화장품/미용 10월 [Sum of AD rank] = {cat_val_8_10["Sum of AD rank"].mean()}')
print(f'화장품/미용 11월 [Sum of AD rank] = {cat_val_8_11["Sum of AD rank"].mean()}')
print(f'화장품/미용 12월 [Sum of AD rank] = {cat_val_8_12["Sum of AD rank"].mean()}')
print(f'화장품/미용 1월 [Sum of AD rank] = {cat_val_8_1["Sum of AD rank"].mean()}')

print('-'*50)

print(f'10월 화장품/미용 [CCC] : {(cat_val_8_10["Click"].mean()) / (cat_val_8_10["Cost"].mean())}')
print(f'12월 화장품/미용 [CCC] : {(cat_val_8_11["Click"].mean()) / (cat_val_8_11["Cost"].mean())}')
print(f'12월 화장품/미용 [CCC] : {(cat_val_8_12["Click"].mean()) / (cat_val_8_12["Cost"].mean())}')

In [ ]:
# 수치표현
# 그래프표현
# text = 월별 패션잡화
print(f'패션잡화 10월 [Click] = {cat_val_9_10["Click"].mean()}')
print(f'패션잡화 11월 [Click] = {cat_val_9_11["Click"].mean()}')
print(f'패션잡화 12월 [Click] = {cat_val_9_12["Click"].mean()}')
print(f'패션잡화 1월 [Click] = {cat_val_9_1["Click"].mean()}')

print('-'*50)

print(f'패션잡화 10월 [Cost] = {cat_val_9_10["Cost"].mean()}')
print(f'패션잡화 11월 [Cost ] = {cat_val_9_11["Cost"].mean()}')
print(f'패션잡화 12월 [Cost] = {cat_val_9_12["Cost"].mean()}')
print(f'패션잡화 1월 [Cost] = {cat_val_9_1["Cost"].mean()}')

print('-'*50)

print(f'패션잡화 10월 [Impression] = {cat_val_9_10["Impression"].mean()}')
print(f'패션잡화 11월 [Impression] = {cat_val_9_11["Impression"].mean()}')
print(f'패션잡화 12월 [Impression] = {cat_val_9_12["Impression"].mean()}')
print(f'패션잡화 1월 [Impression] = {cat_val_9_1["Impression"].mean()}')

print('-'*50)

print(f'패션잡화 10월 [Sum of AD rank] = {cat_val_9_10["Sum of AD rank"].mean()}')
print(f'패션잡화 11월 [Sum of AD rank] = {cat_val_9_11["Sum of AD rank"].mean()}')
print(f'패션잡화 12월 [Sum of AD rank] = {cat_val_9_12["Sum of AD rank"].mean()}')
print(f'패션잡화 1월 [Sum of AD rank] = {cat_val_9_1["Sum of AD rank"].mean()}')

print('-'*50)

print(f'10월 패션잡화 [CCC] : {(cat_val_9_10["Click"].mean()) / (cat_val_9_10["Cost"].mean())}')
print(f'12월 패션잡화 [CCC] : {(cat_val_9_11["Click"].mean()) / (cat_val_9_11["Cost"].mean())}')
print(f'12월 패션잡화 [CCC] : {(cat_val_9_12["Click"].mean()) / (cat_val_9_12["Cost"].mean())}')

In [ ]:
# 수치표현
# 그래프표현
# text = 월별 여가/생활편의
print(f'여가/생활편의 10월 [Click] = {cat_val_10_10["Click"].mean()}')
# print(f'여가/생활편의 11월 [Click] = {cat_val_10_11["Click"].mean()}')
# print(f'여가/생활편의 12월 [Click] = {cat_val_10_12["Click"].mean()}')
# print(f'여가/생활편의 1월 [Click] = {cat_val_10_1["Click"].mean()}')

print('-'*50)

print(f'여가/생활편의 10월 [Cost] = {cat_val_10_10["Cost"].mean()}')
# print(f'여가/생활편의 11월 [Cost ] = {cat_val_10_11["Cost"].mean()}')
# print(f'여가/생활편의 12월 [Cost] = {cat_val_10_12["Cost"].mean()}')
# print(f'여가/생활편의 1월 [Cost] = {cat_val_10_1["Cost"].mean()}')

print('-'*50)

print(f'여가/생활편의 10월 [Impression] = {cat_val_10_10["Impression"].mean()}')
# print(f'여가/생활편의 11월 [Impression] = {cat_val_10_11["Impression"].mean()}')
# print(f'여가/생활편의 12월 [Impression] = {cat_val_10_12["Impression"].mean()}')
# print(f'여가/생활편의 1월 [Impression] = {cat_val_10_1["Impression"].mean()}')

print('-'*50)

print(f'여가/생활편의 10월 [Sum of AD rank] = {cat_val_10_10["Sum of AD rank"].mean()}')
# print(f'여가/생활편의 11월 [Sum of AD rank] = {cat_val_10_11["Sum of AD rank"].mean()}')
# print(f'여가/생활편의 12월 [Sum of AD rank] = {cat_val_10_12["Sum of AD rank"].mean()}')
# print(f'여가/생활편의 1월 [Sum of AD rank] = {cat_val_10_1["Sum of AD rank"].mean()}')

print('-'*50)

print(f'10월 여가/생활편의 [CCC] : {(cat_val_10_10["Click"].mean()) / (cat_val_10_10["Cost"].mean())}')
# print(f'12월 여가/생활편의 [CCC] : {(cat_val_10_11["Click"].mean()) / (cat_val_10_11["Cost"].mean())}')
# print(f'12월 여가/생활편의 [CCC] : {(cat_val_1_12["Click"].mean()) / (cat_val_1_12["Cost"].mean())}')